In [1]:
import openai
import os

In [2]:
import minsearch

In [3]:
import json

In [4]:
from openai import OpenAI

In [5]:
api_key = os.getenv("OPENAI_API_KEY")

In [6]:
client = OpenAI(api_key = api_key)

In [7]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
index = minsearch.Index(
    text_fields=["question", "text"],
    keyword_fields=["course"]
)

In [9]:
index.fit(documents)

In [10]:
# response = client.chat.completions.create(
#     model = 'gpt-4o',
#     messages = [{'role' : 'user', 'content': 'is it too late to join the course?'}])

In [11]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [12]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [47]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
            print(f"Score: {hit['_score']:.2f} | Question: {hit['_source'].get('question')}")
    return result_docs

In [14]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [15]:
from elasticsearch import Elasticsearch

In [16]:
es_client = Elasticsearch('http://localhost:9200') 

In [18]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "homework"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework'})

In [19]:
from tqdm.auto import tqdm


In [20]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [48]:

query = 'How do execute a command on a Kubernetes pod?'

In [49]:
result = rag(query)

Score: 44.51 | Question: How do I debug a docker container?
Score: 35.43 | Question: Kubernetes-dashboard
Score: 33.71 | Question: How do I copy files from a different folder into docker container’s working directory?
Score: 33.26 | Question: How to run a script while a web-server is working?
Score: 32.59 | Question: How can I annotate a graph?


In [50]:
result

'To execute a command on a Kubernetes pod, you would typically use the `kubectl exec` command followed by the pod name and the command you wish to run. Here is the general syntax:\n\n```sh\nkubectl exec [pod-name] -- [command]\n```\n\nIf you need to specify a container within the pod, you can use the `-c` option like this:\n\n```sh\nkubectl exec [pod-name] -c [container-name] -- [command]\n```\n\nAdditionally, if you need an interactive shell session, you can add the `-it` option:\n\n```sh\nkubectl exec -it [pod-name] -- [command]\n```\n\nExample for accessing a shell in a specific pod:\n\n```sh\nkubectl exec -it my-pod -- /bin/sh\n```\n\nThis will open an interactive shell session inside the specified pod.'

In [59]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
            result_docs.append(hit['_source'])
            print(f"Score: {hit['_score']:.2f} | Question: {hit['_source'].get('question')}")
    return result_docs

In [60]:
query = 'How do copy a file to a Docker container?'

In [61]:
result = rag(query)

Score: 73.39 | Question: How do I debug a docker container?
Score: 66.69 | Question: How do I copy files from my local machine to docker container?
Score: 59.81 | Question: How do I copy files from a different folder into docker container’s working directory?


In [62]:
result

'To copy a file from your local machine to a Docker container, you can use the `docker cp` command. The basic syntax for copying a file or directory into a running Docker container is:\n\n```bash\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```'

In [63]:
def build_prompt_q5(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = "\n\n".join([
        context_template.format(question=doc["question"], text=doc["text"])
        for doc in search_results
    ])

    prompt = prompt_template.format(question=query, context=context)
    return prompt

In [65]:
results = elastic_search(query)
prompt = build_prompt_q5(query, results)  

Score: 73.39 | Question: How do I debug a docker container?
Score: 66.69 | Question: How do I copy files from my local machine to docker container?
Score: 59.81 | Question: How do I copy files from a different folder into docker container’s working directory?


In [74]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container

In [66]:
len(prompt)

1446

In [69]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 32.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [71]:
import tiktoken

In [73]:

encoding = tiktoken.encoding_for_model("gpt-4o")


num_tokens = len(encoding.encode(prompt))
print("Token count:", num_tokens)

Token count: 320
